# Introduction
Determin Porygon DT of different user locations to each restaurant in norway city 202


# Data collection
We get the data from big query.


In [8]:
# load packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.io import gbq
from google.cloud import bigquery
# from google.colab import drive # to allow colab save file in my drive
drive.mount('drive')
cwd=os.getcwd()
cwd

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


'/content'

In [0]:
query="""
select * from temp.results_no
"""
# run the query and save the results as data frame named df
df=gbq.read_gbq(query,project_id="fulfillment-dwh-production")


In [10]:
# to save money, next time we can read the csv file instead of query from bigquery
#df.to_csv("taiwan_geohash7.csv", sep="\t")
#!cp taiwan_geohash7.csv "drive/My Drive/dataset"
#df = pd.read_csv("/content/taiwan_geohash7.csv")
df.head()

,user_zone_id,geohash,geohash_center,restaurant_id,weight,porygon_dt
0,9,ukmkmrm,POINT(18.8710784912109 69.693832397461),n8lk,1.0,4
1,9,ukmkmrq,POINT(18.8724517822266 69.693832397461),n8lk,2.0,4
2,9,ukms2r7,POINT(19.0001678466797 69.693832397461),n7my,4.0,4
3,9,ukms2rk,POINT(19.0015411376953 69.693832397461),n7my,5.0,4
4,9,ukms2rm,POINT(19.0029144287109 69.693832397461),n7my,2.0,4


In [15]:
pivot=df.pivot_table("restaurant_id", index=['geohash_center',"geohash","weight"], columns='porygon_dt', aggfunc='count')
totaldistinct=len(pivot) # total distinct geohashes
pivot = pivot.fillna(0) # replace NA with 0
pivot.head()

,,porygon_dt,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30
geohash_center,geohash,weight,,,,,,,,,,,,,,,,,,
POINT(18.8463592529297 69.6855926513672),ukmkkvg,1.0,1.0,0.0,0.0,0.0,2.0,2.0,1.0,0.0,4.0,17.0,10.0,3.0,1.0,0.0,2.0,0.0,0.0,2.0
POINT(18.8463592529297 69.6869659423828),ukmkky5,1.0,1.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,2.0,17.0,13.0,2.0,1.0,0.0,2.0,0.0,0.0,2.0
POINT(18.8477325439453 69.6869659423828),ukmkkyh,1.0,1.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,2.0,14.0,17.0,2.0,0.0,0.0,1.0,1.0,0.0,2.0
POINT(18.8477325439453 69.6883392333984),ukmkkyk,1.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,14.0,19.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0
POINT(18.8504791259766 69.6883392333984),ukmkkyq,1.0,1.0,0.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0,15.0,17.0,3.0,1.0,0.0,1.0,1.0,0.0,1.0


In [16]:
pivot_cumsum = pivot.cumsum(axis=1)
pivot_cumsum

,,porygon_dt,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30
geohash_center,geohash,weight,,,,,,,,,,,,,,,,,,
POINT(18.8463592529297 69.6855926513672),ukmkkvg,1.0,1.0,1.0,1.0,1.0,3.0,5.0,6.0,6.0,10.0,27.0,37.0,40.0,41.0,41.0,43.0,43.0,43.0,45.0
POINT(18.8463592529297 69.6869659423828),ukmkky5,1.0,1.0,1.0,1.0,1.0,2.0,5.0,6.0,6.0,8.0,25.0,38.0,40.0,41.0,41.0,43.0,43.0,43.0,45.0
POINT(18.8477325439453 69.6869659423828),ukmkkyh,1.0,1.0,1.0,1.0,1.0,2.0,5.0,6.0,6.0,8.0,22.0,39.0,41.0,41.0,41.0,42.0,43.0,43.0,45.0
POINT(18.8477325439453 69.6883392333984),ukmkkyk,1.0,1.0,1.0,1.0,1.0,1.0,4.0,5.0,5.0,5.0,19.0,38.0,39.0,40.0,40.0,41.0,42.0,43.0,45.0
POINT(18.8504791259766 69.6883392333984),ukmkkyq,1.0,1.0,1.0,1.0,1.0,3.0,5.0,6.0,6.0,6.0,21.0,38.0,41.0,42.0,42.0,43.0,44.0,44.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
POINT(19.0798187255859 69.6883392333984),ukms6nm,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,8.0,18.0,35.0,41.0,41.0,41.0,43.0,45.0,45.0
POINT(19.0798187255859 69.6897125244141),ukms6nt,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,8.0,18.0,28.0,41.0,41.0,41.0,43.0,44.0,45.0
POINT(19.0811920166016 69.6814727783203),ukms6jn,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,5.0,10.0,17.0,37.0,41.0,41.0,41.0,43.0,45.0,45.0


# Weighted version

In [17]:
weight = pivot_cumsum
weight = weight.reset_index()
weight

porygon_dt,geohash_center,geohash,weight,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30
0,POINT(18.8463592529297 69.6855926513672),ukmkkvg,1.0,1.0,1.0,1.0,1.0,3.0,5.0,6.0,6.0,10.0,27.0,37.0,40.0,41.0,41.0,43.0,43.0,43.0,45.0
1,POINT(18.8463592529297 69.6869659423828),ukmkky5,1.0,1.0,1.0,1.0,1.0,2.0,5.0,6.0,6.0,8.0,25.0,38.0,40.0,41.0,41.0,43.0,43.0,43.0,45.0
2,POINT(18.8477325439453 69.6869659423828),ukmkkyh,1.0,1.0,1.0,1.0,1.0,2.0,5.0,6.0,6.0,8.0,22.0,39.0,41.0,41.0,41.0,42.0,43.0,43.0,45.0
3,POINT(18.8477325439453 69.6883392333984),ukmkkyk,1.0,1.0,1.0,1.0,1.0,1.0,4.0,5.0,5.0,5.0,19.0,38.0,39.0,40.0,40.0,41.0,42.0,43.0,45.0
4,POINT(18.8504791259766 69.6883392333984),ukmkkyq,1.0,1.0,1.0,1.0,1.0,3.0,5.0,6.0,6.0,6.0,21.0,38.0,41.0,42.0,42.0,43.0,44.0,44.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,POINT(19.0798187255859 69.6883392333984),ukms6nm,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,8.0,18.0,35.0,41.0,41.0,41.0,43.0,45.0,45.0
1165,POINT(19.0798187255859 69.6897125244141),ukms6nt,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,8.0,18.0,28.0,41.0,41.0,41.0,43.0,44.0,45.0
1166,POINT(19.0811920166016 69.6814727783203),ukms6jn,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,5.0,10.0,17.0,37.0,41.0,41.0,41.0,43.0,45.0,45.0
1167,POINT(19.0811920166016 69.6869659423828),ukms6nn,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,5.0,10.0,20.0,38.0,41.0,41.0,42.0,43.0,45.0,45.0


In [18]:
cols = list(weight)
del cols[0:3]
cols

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 30]

In [19]:
numOfRestaurants = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
drivingTimesinMin = cols
resArray = []
for indexNumOfRestaurants in numOfRestaurants:
    sumByDrivingTimes = []
    for indexDrivingTimesInMin in drivingTimesinMin:
        sumByDrivingTimes.append(weight.loc[weight[indexDrivingTimesInMin] >= indexNumOfRestaurants, "weight"].sum())
        
    sumByDrivingTimes.insert(0,indexNumOfRestaurants)
    resArray.append(sumByDrivingTimes)

resArray

[[1,
  1820.0,
  2189.0,
  2347.0,
  2410.0,
  2423.0,
  2426.0,
  2427.0,
  2428.0,
  2428.0,
  2428.0,
  2428.0,
  2428.0,
  2429.0,
  2430.0,
  2432.0,
  2432.0,
  2432.0,
  2432.0],
 [5,
  1032.0,
  1450.0,
  1753.0,
  1976.0,
  2093.0,
  2212.0,
  2279.0,
  2367.0,
  2411.0,
  2419.0,
  2425.0,
  2428.0,
  2428.0,
  2428.0,
  2428.0,
  2428.0,
  2429.0,
  2432.0],
 [10,
  671.0,
  1026.0,
  1404.0,
  1633.0,
  1871.0,
  2070.0,
  2137.0,
  2192.0,
  2327.0,
  2405.0,
  2418.0,
  2422.0,
  2424.0,
  2426.0,
  2428.0,
  2428.0,
  2428.0,
  2432.0],
 [15,
  575.0,
  921.0,
  1282.0,
  1572.0,
  1749.0,
  1989.0,
  2115.0,
  2144.0,
  2222.0,
  2318.0,
  2412.0,
  2416.0,
  2419.0,
  2422.0,
  2422.0,
  2426.0,
  2428.0,
  2432.0],
 [20,
  467.0,
  796.0,
  1169.0,
  1451.0,
  1696.0,
  1924.0,
  2085.0,
  2133.0,
  2170.0,
  2276.0,
  2393.0,
  2413.0,
  2416.0,
  2416.0,
  2416.0,
  2424.0,
  2424.0,
  2429.0],
 [25,
  319.0,
  662.0,
  1015.0,
  1354.0,
  1601.0,
  1827.0,
  2032.0

In [20]:
colNames = np.append("Number of Restaurants", drivingTimesinMin)

df2 = pd.DataFrame(resArray,
                   columns = colNames)
df2=df2.set_index('Number of Restaurants')
df2

,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30
Number of Restaurants,,,,,,,,,,,,,,,,,,
1,1820.0,2189.0,2347.0,2410.0,2423.0,2426.0,2427.0,2428.0,2428.0,2428.0,2428.0,2428.0,2429.0,2430.0,2432.0,2432.0,2432.0,2432.0
5,1032.0,1450.0,1753.0,1976.0,2093.0,2212.0,2279.0,2367.0,2411.0,2419.0,2425.0,2428.0,2428.0,2428.0,2428.0,2428.0,2429.0,2432.0
10,671.0,1026.0,1404.0,1633.0,1871.0,2070.0,2137.0,2192.0,2327.0,2405.0,2418.0,2422.0,2424.0,2426.0,2428.0,2428.0,2428.0,2432.0
15,575.0,921.0,1282.0,1572.0,1749.0,1989.0,2115.0,2144.0,2222.0,2318.0,2412.0,2416.0,2419.0,2422.0,2422.0,2426.0,2428.0,2432.0
20,467.0,796.0,1169.0,1451.0,1696.0,1924.0,2085.0,2133.0,2170.0,2276.0,2393.0,2413.0,2416.0,2416.0,2416.0,2424.0,2424.0,2429.0
25,319.0,662.0,1015.0,1354.0,1601.0,1827.0,2032.0,2119.0,2151.0,2238.0,2358.0,2407.0,2412.0,2413.0,2414.0,2419.0,2422.0,2429.0
30,79.0,532.0,846.0,1207.0,1459.0,1762.0,1982.0,2096.0,2137.0,2194.0,2317.0,2397.0,2404.0,2407.0,2409.0,2416.0,2422.0,2427.0
35,0.0,170.0,601.0,974.0,1264.0,1577.0,1847.0,2035.0,2111.0,2148.0,2267.0,2381.0,2394.0,2401.0,2403.0,2409.0,2416.0,2425.0
40,0.0,0.0,7.0,317.0,765.0,1123.0,1337.0,1623.0,1964.0,2072.0,2152.0,2266.0,2342.0,2355.0,2363.0,2387.0,2403.0,2421.0


In [22]:
totalnum = weight["weight"].sum()
df2[["4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20", "30"]].div(totalnum)

,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30
Number of Restaurants,,,,,,,,,,,,,,,,,,
1,0.748355,0.900082,0.965049,0.990954,0.996299,0.997533,0.997944,0.998355,0.998355,0.998355,0.998355,0.998355,0.998766,0.999178,1.000000,1.000000,1.000000,1.000000
5,0.424342,0.596217,0.720806,0.812500,0.860609,0.909539,0.937089,0.973273,0.991365,0.994655,0.997122,0.998355,0.998355,0.998355,0.998355,0.998355,0.998766,1.000000
10,0.275905,0.421875,0.577303,0.671464,0.769326,0.851151,0.878701,0.901316,0.956826,0.988898,0.994243,0.995888,0.996711,0.997533,0.998355,0.998355,0.998355,1.000000
15,0.236431,0.378701,0.527138,0.646382,0.719161,0.817845,0.869655,0.881579,0.913651,0.953125,0.991776,0.993421,0.994655,0.995888,0.995888,0.997533,0.998355,1.000000
20,0.192023,0.327303,0.480674,0.596628,0.697368,0.791118,0.857319,0.877056,0.892270,0.935855,0.983964,0.992188,0.993421,0.993421,0.993421,0.996711,0.996711,0.998766
25,0.131168,0.272204,0.417352,0.556743,0.658306,0.751234,0.835526,0.871299,0.884457,0.920230,0.969572,0.989720,0.991776,0.992188,0.992599,0.994655,0.995888,0.998766
30,0.032484,0.218750,0.347862,0.496299,0.599918,0.724507,0.814967,0.861842,0.878701,0.902138,0.952714,0.985609,0.988487,0.989720,0.990543,0.993421,0.995888,0.997944
35,0.000000,0.069901,0.247122,0.400493,0.519737,0.648438,0.759457,0.836760,0.868010,0.883224,0.932155,0.979030,0.984375,0.987253,0.988076,0.990543,0.993421,0.997122
40,0.000000,0.000000,0.002878,0.130345,0.314556,0.461760,0.549753,0.667352,0.807566,0.851974,0.884868,0.931743,0.962993,0.968339,0.971628,0.981497,0.988076,0.995477


## Non-weighted version
Here we use another way to replicate the table.


In [23]:
result = pivot_cumsum.apply(pd.value_counts).fillna(0)
result["index"]=result.index
result

porygon_dt,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30,index
0.0,367.0,158.0,53.0,15.0,7.0,6.0,5.0,4.0,4.0,4.0,4.0,4.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0
1.0,206.0,240.0,208.0,144.0,78.0,34.0,3.0,2.0,2.0,2.0,0.0,0.0,1.0,2.0,3.0,3.0,3.0,0.0,1.0
2.0,101.0,96.0,99.0,84.0,76.0,38.0,38.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
3.0,35.0,57.0,38.0,43.0,36.0,38.0,22.0,14.0,8.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4.0,56.0,22.0,19.0,15.0,29.0,27.0,34.0,10.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0
5.0,88.0,55.0,36.0,24.0,19.0,19.0,18.0,8.0,14.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
6.0,34.0,83.0,47.0,37.0,30.0,32.0,36.0,29.0,14.0,0.0,0.0,3.0,2.0,1.0,0.0,0.0,1.0,0.0,6.0
7.0,19.0,34.0,71.0,63.0,24.0,27.0,19.0,28.0,10.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0
8.0,14.0,20.0,27.0,48.0,31.0,17.0,22.0,40.0,14.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
9.0,14.0,13.0,4.0,11.0,17.0,10.0,4.0,13.0,11.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0


In [0]:
column1=result[result["index"]>=1].sum()
column5=result[result["index"]>=5].sum()
column10=result[result["index"]>=10].sum()
column15=result[result["index"]>=15].sum()
column20=result[result["index"]>=20].sum()
column25=result[result["index"]>=25].sum()
column30=result[result["index"]>=30].sum()
column35=result[result["index"]>=35].sum()
column40=result[result["index"]>=40].sum()
column45=result[result["index"]>=45].sum()
column50=result[result["index"]>=50].sum()
aggregate=pd.concat([column1, column5, column10, column15, column20, column25, column30, column35, column40, column45, column50], axis=1)
aggregate.columns = [1,5,10,15,20,25,30, 35, 40, 45, 50]
aggregate=aggregate.T

In [26]:
aggregate[drivingTimesinMin].div(totaldistinct)

porygon_dt,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30
1,0.686056,0.864842,0.954662,0.987169,0.994012,0.994867,0.995723,0.996578,0.996578,0.996578,0.996578,0.996578,0.997434,0.998289,1.000000,1.000000,1.000000,1.000000
5,0.345595,0.509837,0.643285,0.742515,0.806672,0.877673,0.912746,0.958084,0.987169,0.991446,0.994012,0.996578,0.996578,0.996578,0.996578,0.996578,0.997434,1.000000
10,0.201027,0.334474,0.485030,0.585971,0.703165,0.787853,0.828058,0.857143,0.933276,0.982891,0.991446,0.993157,0.994012,0.994867,0.996578,0.996578,0.996578,1.000000
15,0.168520,0.292558,0.430282,0.555175,0.648417,0.743370,0.812660,0.834046,0.875107,0.929855,0.988024,0.990590,0.991446,0.993157,0.993157,0.995723,0.996578,1.000000
20,0.130881,0.248075,0.381523,0.507271,0.615911,0.716852,0.796407,0.827203,0.847733,0.907613,0.973482,0.988024,0.990590,0.990590,0.990590,0.994867,0.994867,0.997434
25,0.071856,0.198460,0.333618,0.467921,0.577417,0.675791,0.765612,0.816938,0.839179,0.887083,0.951240,0.984602,0.987169,0.988024,0.988879,0.991446,0.993157,0.997434
30,0.018820,0.151411,0.268606,0.411463,0.520958,0.641574,0.737382,0.800684,0.828914,0.862275,0.925577,0.976903,0.982036,0.984602,0.985458,0.988879,0.993157,0.996578
35,0.000000,0.032506,0.168520,0.314799,0.443969,0.571429,0.675791,0.769889,0.810950,0.836612,0.899059,0.964072,0.974337,0.979470,0.981180,0.985458,0.988879,0.994867
40,0.000000,0.000000,0.002566,0.096664,0.238666,0.370402,0.476476,0.603080,0.733105,0.791275,0.836612,0.893071,0.942686,0.952096,0.958939,0.972626,0.981180,0.991446
45,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000855,0.028229,0.076133,0.224979,0.348161,0.429427,0.560308,0.651839,0.740804,0.856287,0.965783


## Kepler Visualisation


In [0]:
# %%capture help hide the installing process
%%capture   
!pip install keplergl

In [0]:
from keplergl import KeplerGl 

In [0]:
kepler = pivot.reset_index()
kepler = kepler.fillna(0)
kepler

porygon_dt,geohash_center,geohash,weight,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,POINT(121.343307495117 25.0440216064453),wsqq3pp,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,4.0,10.0,16.0,31.0,34.0,40.0,25.0,24.0,14.0
1,POINT(121.343307495117 25.0838470458984),wsqq9nr,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,4.0,11.0,23.0,39.0,25.0,8.0,11.0,26.0,43.0
2,POINT(121.343307495117 25.0865936279297),wsqq9nz,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,5.0,15.0,25.0,35.0,23.0,7.0,12.0,26.0
3,POINT(121.344680786133 25.0399017333984),wsqq3q2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,7.0,14.0,6.0,31.0,42.0,39.0,43.0,5.0,7.0
4,POINT(121.344680786133 25.0412750244141),wsqq3q8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,7.0,14.0,7.0,35.0,41.0,42.0,30.0,9.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14118,POINT(121.674270629883 25.0824737548828),wsqw8y0,6.0,0.0,0.0,2.0,0.0,2.0,0.0,6.0,17.0,3.0,18.0,18.0,16.0,25.0,8.0,12.0,4.0,7.0,3.0,7.0
14119,POINT(121.675643920898 25.0797271728516),wsqw8v9,10.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,6.0,14.0,9.0,18.0,17.0,16.0,24.0,11.0,7.0,6.0,7.0,2.0
14120,POINT(121.675643920898 25.0811004638672),wsqw8vc,16.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,9.0,15.0,8.0,20.0,15.0,15.0,24.0,11.0,8.0,3.0,8.0,3.0
14121,POINT(121.675643920898 25.0838470458984),wsqw8y3,2.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,11.0,9.0,13.0,19.0,12.0,23.0,19.0,9.0,10.0,5.0,5.0,3.0


In [0]:
col_list = list(kepler)
del col_list[0:3]
col_list

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [0]:
kepler["total_restaurant"]=kepler[col_list].sum(axis=1)
kepler

porygon_dt,geohash_center,geohash,weight,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,total_restaurant
0,POINT(121.343307495117 25.0440216064453),wsqq3pp,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,4.0,10.0,16.0,31.0,34.0,40.0,25.0,24.0,14.0,204.0
1,POINT(121.343307495117 25.0838470458984),wsqq9nr,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,4.0,11.0,23.0,39.0,25.0,8.0,11.0,26.0,43.0,197.0
2,POINT(121.343307495117 25.0865936279297),wsqq9nz,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,5.0,15.0,25.0,35.0,23.0,7.0,12.0,26.0,154.0
3,POINT(121.344680786133 25.0399017333984),wsqq3q2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,7.0,14.0,6.0,31.0,42.0,39.0,43.0,5.0,7.0,196.0
4,POINT(121.344680786133 25.0412750244141),wsqq3q8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,7.0,14.0,7.0,35.0,41.0,42.0,30.0,9.0,15.0,204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14118,POINT(121.674270629883 25.0824737548828),wsqw8y0,6.0,0.0,0.0,2.0,0.0,2.0,0.0,6.0,17.0,3.0,18.0,18.0,16.0,25.0,8.0,12.0,4.0,7.0,3.0,7.0,148.0
14119,POINT(121.675643920898 25.0797271728516),wsqw8v9,10.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,6.0,14.0,9.0,18.0,17.0,16.0,24.0,11.0,7.0,6.0,7.0,2.0,141.0
14120,POINT(121.675643920898 25.0811004638672),wsqw8vc,16.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,9.0,15.0,8.0,20.0,15.0,15.0,24.0,11.0,8.0,3.0,8.0,3.0,143.0
14121,POINT(121.675643920898 25.0838470458984),wsqw8y3,2.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,11.0,9.0,13.0,19.0,12.0,23.0,19.0,9.0,10.0,5.0,5.0,3.0,142.0


In [0]:
!pip install pydeck

In [0]:
import pydeck

In [0]:
map_1 = KeplerGl()
#map_1.add_data(data=kepler, name='data_1')
!pip install ipython

import IPython

IPython.display.HTML(map_1)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


TypeError: ignored

In [0]:
map_1

KeplerGl()